<a href="https://colab.research.google.com/github/tessamitchell/SchoolBusStopSignDetection/blob/main/FinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports & Data Set Up

In [ ]:
# Load libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from skimage.feature import local_binary_pattern
from skimage.feature import haar_like_feature
from skimage.feature import hog

import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import xml.etree.ElementTree as ET

In [ ]:
# X=data
# y=data.labels

!pip install roboflow
from roboflow import Roboflow
from google.colab import userdata

rf = Roboflow(api_key=userdata.get('roboflow'))
project = rf.workspace("myworkspace-hr4qa").project("stop-sign-zn1kw-r77i3")
version = project.version(1)
dataset = version.download("voc")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 24.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Stop-Sign-1 in voc:: 100%|██████████| 3196/3196 [00:00<00:00, 6801.78it/s]


In [ ]:
# https://chatgpt.com/share/692615ef-8958-8010-adfd-24ddd028c3e9
def load_voc_annotation(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    boxes = []
    for obj in root.findall("object"):
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])

    return boxes

In [ ]:
import glob
xml_paths = glob.glob("/content/your_dataset_name-1/train/*.xml")
# xml_paths = glob.glob("/content/*/*.xml", recursive=True) # for train valid and test xml

In [ ]:
images=[]
neg_images=[]

# Preprocessing

# Feature Extraction

[sci-kit lbp extraction](https://scikit-image.org/docs/0.25.x/auto_examples/features_detection/plot_local_binary_pattern.html)

In [ ]:
# https://scikit-image.org/docs/0.25.x/auto_examples/features_detection/plot_local_binary_pattern.html

radius=3
n_points=8*radius
METHOD = 'uniform'
def extract_lbp(img):
  res=local_binary_pattern(img,n_points,radius,METHOD)
  return res


[scikit haar face extraction](https://scikit-image.org/docs/0.25.x/auto_examples/applications/plot_haar_extraction_selection_classification.html)

[haar general additional link](https://scikit-image.org/docs/0.25.x/auto_examples/features_detection/plot_haar.html)

In [ ]:
feature_types = ['type-2-x', 'type-2-y', 'type-3-x', 'type-3-y', 'type-4']
def extract_haar(img):
  haar=haar_like_feature(img,0,0,img.shape[0],img.shape[1])
  return haar

In [ ]:
def extract_hog(img):
  feature,vis=hog(img,orientations=9, pixels_per_cell=(8, 8),cells_per_block=(2, 2))
  return feature

In [ ]:
def extract_features(img):
  img=img[:,:,2] # extract red color channel, assume OpenCV BGR formatting
  return np.concatenate([extract_lpb(img),extract_haar(img),extract_hog(img)])

In [ ]:
X_features=[]
y_labels=[]
for window in images:
  X_features.append(extract_features(window))
  y_labels.append(1)

for window in neg_images:
  X_features.append(extract_features(window))
  y_labels.append(0)

# Training (Adaboost Cascade)

Adding these links here for any tabs I opened in my research that I might want to come back to

[sklearn adaboost tutorial](https://www.datacamp.com/tutorial/adaboost-classifier-python)

[another adaboost tutorial](https://www.kdnuggets.com/2022/10/implementing-adaboost-scikitlearn.html)

[sklearn adaboost documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)

[adaboost from scratch](https://medium.com/@enozeren/building-the-adaboost-model-from-scratch-with-python-db3a8a763484)

[from scratch G4G](https://www.geeksforgeeks.org/machine-learning/implementing-the-adaboost-algorithm-from-scratch/)

[sklearn pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X_features, y_labels, test_size=0.3,random_state=42) # 70% training and 30% test

In [ ]:
# Create adaboost classifer object
abc = AdaBoostClassifier(n_estimators=50,
                         learning_rate=1)
# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Extract Regions of Interest

In [ ]:
def extract_windows(img):
  return x,y,w,h

In [ ]:
testimages=[]
positive_windows=[]
for img in testimages:
  windows=extract_windows(img)
  for (x,y,w,h) in windows:
    patch=img[x:x+h,y:y+w]
    features=extract_features(patch)
    if model.predict(features)==1:
      positive_windows.append((img,patch,x,y,w,h))

# Light Detection (Hough Circle Transform)

### Hough Circle Transform

In [ ]:
# documentation https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga47849c3be0d0406ad3ca45db65a25d2d
def houghAndValidation(img):
  blurred=cv.medianBlur(img,5)
  minR=blurred.shape[1]/5
  maxR=blurred.shape[1]/3
  circles = cv.HoughCircles(blurred,cv.HOUGH_GRADIENT,1,20,
                              param1=50,param2=30,minRadius=minR,maxRadius=maxR)

  circles = np.uint16(np.around(circles))
  # validation

  for c in circles:
    # compare radius with bounding box of stop sign

    # get average intensity value and compare to red section of stop sign





In [ ]:
for win in positive_windows:
  cw=canny(win[1])
  hw=houghAndValidation(cw)


# Outputs

In [ ]:
# https://docs.opencv.org/4.x/dc/da5/tutorial_py_drawing_functions.html
def drawBounds(img,circle,box):
  cv.circle(img,(circle[0],circle[1]),circle[2],(255,0,0),2)
  cv.rectangle(img,(box[0],box[1]),(box[2],box[3]),(0,255,0),2)